In [ ]:
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

In [ ]:
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import gensim
from gensim import utils
from nltk import word_tokenize
from nltk import download

from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [ ]:
word_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)


In [ ]:
def document_vector(word2vec_model, doc):
    doc = [word for word in doc if word in word2vec_model.vocab]
    if(len(doc)==0):
      return [0]*300
    return np.mean(word2vec_model[doc], axis=0)


In [ ]:
download('punkt')

In [ ]:
file1 = open('MR.x.train.txt', 'r',encoding='ISO-8859-1') 
Lines = file1.readlines() 
Lines[0]

In [ ]:
file1 = open('MR.x.test.txt', 'r',encoding='ISO-8859-1') 
Lines3 = file1.readlines() 
Lines3[0]

In [ ]:
list_of_sentence=[]
preprocessed_texts=[lines.lower() for lines in Lines]
for j in range(len(preprocessed_texts)):
  temp=[] 
  for chars in range(2,15):
    for i in range(len(preprocessed_texts[j])-chars):
      seq = str(preprocessed_texts[j][i:i+chars])
      temp.append(seq)
  list_of_sentence.append(temp)
n=len(list_of_sentence)

In [ ]:
list_of_sentence[0]

In [ ]:
list_of_sentence3=[]
preprocessed_texts3=[lines.lower() for lines in Lines3]
for j in range(len(preprocessed_texts3)):
  temp=[] 
  for chars in range(2,15):
    for i in range(len(preprocessed_texts3[j])-chars):
      seq = str(preprocessed_texts3[j][i:i+chars])
      temp.append(seq)
  list_of_sentence3.append(temp)
n=len(list_of_sentence3)

In [ ]:
from sklearn.preprocessing import LabelEncoder

file1 = open('MR.y.train.txt', 'r') 
Lines = file1.readlines() 
count = 0
target1=[]
for line in Lines: 
    target1.append(line)
le = LabelEncoder()
le.fit(["NEG\n", "POS\n"])
list(le.classes_)
labels=le.transform(target1)
labels[:29]

In [ ]:
pos=0
for i in labels:
  if(i==0):
    pos+=1
print(pos)

In [ ]:
corpus=list_of_sentence
texts=Lines
y=labels

In [ ]:
corpus3=list_of_sentence3
texts3=Lines3

In [ ]:
x =[]
for doc in corpus: 
    x.append(document_vector(word_model, doc))
X = np.array(x) 
X.shape, len(y)


In [ ]:
x3 =[]
for doc in corpus3: 
    x3.append(document_vector(word_model, doc))
X3 = np.array(x3)           #
X3.shape


In [ ]:
import tensorflow as tf

In [ ]:
train_x=X
train_y=y
X_train, X_val, Y_train, Y_val = train_test_split(train_x, train_y, test_size=0.20, random_state=42)
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
Y_train=tf.convert_to_tensor(Y_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
Y_val=tf.convert_to_tensor(Y_val, dtype=tf.float32)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=6, max_iter=1000,random_state=0,multi_class='ovr').fit(X_train, Y_train)
score = clf.score(X_val, Y_val)

print("Accuracy:", score)

In [ ]:
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
scale.fit(X_train)
X_train=scale.transform(X_train)
X_val=scale.transform(X_val)
train_x=scale.transform(train_x)


In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000,random_state=0)
classifier.fit(X_train, Y_train)
score = classifier.score(X_val, Y_val)

print("Accuracy:", score)

In [ ]:
y_pred=classifier.predict(X3)

In [ ]:
print(y_pred[:20])

In [ ]:
pos=0
for i in y_pred:
  if(i==1):
    pos+=1
print(pos)

In [ ]:
model = Sequential()
model.add(Dense(300, input_dim=300, activation='linear'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history=model.fit(X_train, Y_train, validation_data=(X_val,Y_val), epochs=5, batch_size=128)

In [ ]:
y_pred2=model.predict_classes(X3)

In [ ]:
pos=0
for i in y_pred2:
  if(i==1):
    pos+=1
print(pos)

In [ ]:
print(y_pred2[:12])

In [ ]:
score=model.evaluate(X_val,Y_val)

In [ ]:
Y_test=[]
for i in range(len(y_pred)):
  if(y_pred[i]==0):
    Y_test.append('NEG')
    continue
  if(y_pred[i]==1):
    Y_test.append('POS')
    continue


In [ ]:
f1 = open("char2.MR.y.test.txt", "w")
for y in Y_test:
  y1=str(y)
  f1.write(str(y1))
  f1.write("\n")

In [ ]:
f2=open("char2.MR.y.test.txt","r")
for x in f2:
  print(x)